# SNR Calculation Application
This is an application of my simulation environment to predict SNR values for the Lake Wheeler Road AERPAW Environment. The goal is to train a model or construct a lookup table so we can find SNR values from arbitrary position inside the Phase 1 Geofence to each of the 4 Base Stations (LW1-LW4). These SNR values will be used in the model we are deploying in the AERPAW AADM challenge.

In [1]:
import numpy as np
import sys
sys.path.append("..")
from src.EnvironmentFramework import Environment, UAV, GroundUser

# Computing the Bounding Box
LAUNCH = np.array([35.72752574530495, -78.69616739508318, 25])
DELTA_LAT = 0.02  # Half of the bounding box height, in degrees
DELTA_LON = 0.02 # Half of the bounding box width, in degrees
BBOX = np.array([
    [LAUNCH[0] - DELTA_LAT, LAUNCH[0] + DELTA_LAT],
    [LAUNCH[1] - DELTA_LON, LAUNCH[1] + DELTA_LON]
])

# Guarantees a square bounding box with the launch point at the center
# This is min/max latitude, and then min/max longitude to input into the Blender OSM Import
print(BBOX)


[[ 35.70752575  35.74752575]
 [-78.7161674  -78.6761674 ]]


In [2]:
# Creating the environment, takes a while
env = Environment("C:/Users/legoe/Blender/BlenderDataFiles/SNR_Calculation/lake_wheeler_road_aerpaw.xml",
                  "C:/Users/legoe/Sumo/2024-10-04-09-05-18/simulated_final_person_new.csv",
                  time_step=1, ped_height=1.5, ped_rx=True, wind_vector=np.zeros(3))
env.setTransmitterArray(AntennaArray(antenna=Antenna("tr38901", "V"), positions=tf.Variable([0.0,0.0,0.0])))
env.setReceiverArray(AntennaArray(antenna=Antenna("tr38901", "V"), positions=tf.Variable([0.0,0.0,0.0])))
env.scene.preview(show_devices=False)

NameError: name 'AntennaArray' is not defined

## Checking Materials
When the data was exported, I tried to specify that the material be medium dry ground according to Sionna specifications. I am not entirely sure if this is the case, so we check it and correct it to the correct material if needed. The material is important because we want to consider ground reflection paths in addition to Line-Of-Sight paths. That is why we need ground topology and material data for Sionna to use for the ray tracing simulation. 

In [ ]:
for object in env.scene._scene_objects:
    env.scene.get(object).radio_material = "itu_medium_dry_ground"

## LatLon <-> Cartesian Conversion Functions
These methods are used to exchange between a local coordinate system used by Sionna, and the spherical coordinate system used by AERPAW. It should assume the launch point as the origin for the local coordinate system. I will have to check the coordinate alignment of the imported Blender model, which was X forward and Z up.

In [ ]:
R = 6371201  # Radius of Earth at Raleigh, in meters

def latLonToCartesian(basis, lat_lon):
    """
    Converts a series of points in lat_lon to points in cartesian with respect to the basis point in lat/lon
    The height, which should be the third element of the points, is preserved

    Args:
        basis (np.array(float)): The lat/lon/height coordinates of the basis point, in degrees and meters
        lat_lon (np.array(np.array(float))): The lat/lon/height coordinates of the points to convert, in degrees and meters

    Returns:
        np.array(np.array(float)): The x/y/z points in cartesian with respect to the basis point, in meters
    """

    rtn = []
    for i in range(len(lat_lon)):
        rtn.append(cartesian(lat_lon[i], basis))
    return np.array(rtn)


def cartesianToLatLon(basis, coordinates):
    rtn = []
    for i in range(len(coordinates)):
        rtn.append(latLon(coordinates[i], basis))
    return np.array(rtn)


def cartesian(lat_lon, basis):
    """
    Converts a single point to lat/lon, with respect to the basis point
    """
    return np.array([R * np.sin((lat_lon[0] - basis[0]) * np.pi / 180), 
                    R * np.sin((lat_lon[1] - basis[1]) * np.pi / 180), 
                    lat_lon[2]])
    

def latLon(coordinate, basis):
    """
    Gets the lat/lon coordinate of a point with respect to the lat/lon basis point

    Args:
        coordinate (np.array()): The cartesian coordinate to convert in meters
        basis (np.array()): The basis coordinate to use as a zero point, in lat/lon

    Returns:
        np.array(): the lat/lon coordinates of the point
    """
    return np.array([np.asin(180 / (np.pi * R) * coordinate[0]) + basis[0],
                     np.asin(180 / (np.pi * R) * coordinate[1]) + basis[1],
                     coordinate[2]])

## Modifying Receivers
We want to get rid of the pedestrians in the scene, because they are not necessary to our current query of SNR data. We want to get our own receivers in the Sionna environment that line up with the positions of the base stations in the lake wheeler road AERPAW environment. This requires a bit of coordinate bashing and conversions but should be easily reconcilable.

In [ ]:
# Ground level is about 37 meters, so add this to the height of the receivers and UAVs
GROUND_HEIGHT = 37

BASE_STATIONS_LATLON = np.array([
    [35.72750947, -78.69595810, 82.973],
    [35.72821305, -78.70090823, 78.947],
    [35.72491205, -78.69190014, 72.345],
    [35.73318358, -78.6983642, 89.345]
])

BASE_STATIONS_LOCAL = latLonToCartesian(LAUNCH, BASE_STATIONS_LATLON)
print(BASE_STATIONS_LOCAL)

# Rotate 180 degrees for offset in the model
for bs in BASE_STATIONS_LOCAL:
    # bs[0] *= -1
    bs[1] *= -1

env.scene._receivers.clear()
base_stations = []
for i in range(len(BASE_STATIONS_LOCAL)):
    BASE_STATIONS_LOCAL[i][2] += GROUND_HEIGHT
    g = GroundUser(i, np.array([BASE_STATIONS_LOCAL[i]]), height=BASE_STATIONS_LOCAL[i][2], com_type="rx", delta_t=1)
    base_stations.append(g)
    env.scene.add(g.device)
    
env.gus = np.array(base_stations)
env.scene.preview(show_devices=True)
# This is verified as the correct orientation using the Blender model and validating it with the surrounding building data from OpenStreetMaps

## Adding UAV Device
We will insert one UAV device, and then move it around the scene to different positions to record the SNR values there. We also need to account for any potential rotations in our import.

In [ ]:
env.addUAV(0, pos=np.array([0, 0, 25 + GROUND_HEIGHT]), bandwidth=300)
env.scene.preview(show_devices=True)

## Calculating SNR
We now wish to set the UAV to a certain local position inside the simulation, and then compute the SNR values from that position to each of the base stations in the Lake Wheeler Road AERPAW environment.

In [ ]:
def computeSNR(uav_position):
    env.moveAbsUAV(0, uav_position, np.zeros(3))
    paths = env.scene.compute_paths(max_depth=2, method="fibonacci", num_samples=1000000, los=True, reflection=True, diffraction=False, scattering=False, check_scene=False)
    a, tau = paths.cir(los=True, reflection=True, diffraction=False, scattering=False, ris=False)
    return a

print(computeSNR(np.array([0, 0, 25 + GROUND_HEIGHT])))